In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [9]:
from langchain import embeddings
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
import langchain.chat_models

In [4]:
!pip install pypdf --quiet

In [7]:
!pip install openai --quiet

In [10]:
!pip install tiktoken --quiet

In [12]:
!pip install faiss-cpu --quiet

In [2]:
import os
#os.environ['OPENAI_API_KEY'] = ''

In [5]:
loader = PyPDFDirectoryLoader("extras/")
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
documents = loader.load()
data = text_splitter.split_documents(documents)


In [8]:
embeddings = OpenAIEmbeddings()

In [13]:
vectorstore = FAISS.from_documents(data, embeddings)
# Persist the vectors locally on disk
vectorstore.save_local("vectorstore/")

In [1]:
import tempfile
import streamlit as st
from langchain.document_loaders import PyMuPDFLoader
from tools.llama2 import loadllama
from tools.googlevertex import loadvertex
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import VertexAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import SequentialChain

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.callbacks.base import BaseCallbackHandler


import os
#os.environ['OPENAI_API_KEY'] = ''


class StreamHandler(BaseCallbackHandler):
    
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text=initial_text
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        # "/" is a marker to show difference 
        # you don't need it 
        self.text+=token+"/" 
        self.container.markdown(self.text) 

class iedge7_RAG:

    def __init__(self):
        
        self.DB_FAISS_PATH = 'vectorstore/'

    def conversational_chat(self,query,top_k,max_o_t,tempr):
        
        DB_FAISS_PATH = 'vectorstore/'

        
        embeddings = OpenAIEmbeddings()
        
        db = FAISS.load_local(DB_FAISS_PATH, embeddings)
        similarity_data = db.similarity_search(query)

        db_s_f = FAISS.from_texts([similarity_data[0].page_content] , embeddings)
        
        for i in range(top_k-1):
            db_s_i = FAISS.from_texts([similarity_data[i+1].page_content] , embeddings)
            db_s_f.merge_from(db_s_i)
        
        retriever = db_s_f.as_retriever()

        db = FAISS.load_local(DB_FAISS_PATH, embeddings)

        retriever = db.as_retriever()

        template = """
                    Use the following context about a University to answer student's question 
                    {context}

                    Student Question: {question}

                    Answer the question in points 
                    
                    """
        prompt = ChatPromptTemplate.from_template(template)
        
        model = ChatOpenAI(temperature = tempr, streaming=True, callbacks=[stream_handler])

        chain = (
                    {"context": retriever, "question": RunnablePassthrough()}
                    | prompt
                    | model
                    | StrOutputParser()
                    )
        answer = chain.invoke(query)
        
        return answer

: 